# lite

In [ ]:
import json
import os
import re

import importnb

with importnb.Notebook():
    from jupyak.tasks import _actions as A
    from jupyak.tasks import _py as P
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

In [ ]:
def lite_tasks(yak: Y.Yak):
    work_path = yak.work_path

    pth_deps = []
    ext_deps = []
    whl_deps = []
    for name, repo in yak.py_repos.items():
        if not repo.lite:
            continue

        pth_deps += [
            yak.env.py_site_packages / P._pth_path(pth) for pth in repo.lite.needs_pth
        ]
        for ext_js_path, ext_info in repo.py.lab_extensions.items():
            ext_deps += [repo.work_path / t for t in ext_info["targets"]]
        if repo.lite.wheel:
            for ppt in repo.py.pyproject_tomls:
                ppt_path = repo.work_path / ppt
                if any(
                    re.match(swp, ppt_path.parent.name)
                    for swp in repo.lite.skip_wheel_patterns
                ):
                    continue
                whl_deps += [ppt_path.parent / "dist" / W.SHA256SUMS]
    lite = yak.lite.work_path
    build_conf = yak.lite.build_config_path

    gist_dep = []

    if yak.lite.gist:
        gist_dep += [yak.lite.gist_path / ".git/refs/heads" / W.WORK_BRANCH]

    yield dict(
        name="config",
        doc="> configure jupyterlite",
        file_dep=[*ext_deps, *whl_deps, *gist_dep],
        actions=[(_make_lite_config, [ext_deps, whl_deps, yak])],
        targets=[build_conf],
    )
    in_work_lite = {"cwd": lite}
    run_args = yak.env.run_args
    lite_args = [*run_args, "jupyter", "lite"]

    yield dict(
        name="build",
        doc="> build lite site archive",
        file_dep=[
            *pth_deps,
            *ext_deps,
            *whl_deps,
            *gist_dep,
            build_conf,
            yak.env.lab_share / "static" / W.PACKAGE_JSON,
        ],
        actions=[
            A.run([*lite_args, "build"], in_work_lite),
            A.run([*lite_args, "archive"], in_work_lite),
        ],
        targets=[yak.lite.app_shasums_path],
    )

In [ ]:
def _make_lite_config(ext_deps, whl_deps, yak: Y.Yak):
    work_path = yak.lite.work_path
    work_path.mkdir(exist_ok=True, parents=True)
    contents = []
    lbc = {}
    pla = {}
    config = {"LiteBuildConfig": lbc, "PipliteAddon": pla}

    if yak.lite.gist:
        contents = [os.path.relpath(yak.lite.gist_path, work_path)]
        gist_config_path = yak.lite.gist_path / W.JUPYTER_LITE_CONFIG
        if gist_config_path.exists():
            gist_config = json.loads(gist_config_path.read_text(encoding="utf-8"))
            for has_traits, trait_values in gist_config.items():
                config.setdefault(has_traits, {}).update(trait_values)
        for run_path in [W.JUPYTER_LITE_JSON, W.JUPYTER_LITE_IPYNB]:
            src = yak.lite.gist_path / run_path
            if not src.exists():
                continue
            dest = work_path / run_path
            dest.write_text(src.read_text(encoding="utf-8"), encoding="utf-8")

    lbc.update(
        cache_dir="../.cache/lite",
        contents=contents,
        output_dir=os.path.relpath(yak.lite.app_path, work_path),
        ignore_sys_prefix=["federated_extensions"],
        federated_extensions=[
            os.path.relpath(pkg_json.parent, work_path)
            for pkg_json in ext_deps
            if pkg_json.name == W.PACKAGE_JSON
        ],
    )

    pla.update(
        piplite_urls=[
            *pla.get("piplite_urls", []),
            *[os.path.relpath(whl_shas.parent, work_path) for whl_shas in whl_deps],
        ]
    )

    yak.lite.build_config_path.write_text(json.dumps(config, indent=2, sort_keys=True))